# AI Co-Performer Demo

This notebook demonstrates **AI as a responsive musical partner** in BrainJam.

Key concepts:
- **Performer → AI**: Your input influences AI timing, density, and character
- **AI → Performer**: AI responds musically (call-and-response patterns)
- **Feedback loop**: You and AI mutually influence each other
- **Not autonomous**: AI follows your lead, doesn't take over

---

## Performance Paradigm

Think of the AI as a **responsive ensemble member**:
- When you're active → AI is active
- When you're sparse → AI fills space
- When you're tense → AI adds tension
- When you're calm → AI provides grounding

**You maintain creative control** through your input signals.

In [ ]:
# Import required libraries
import sys
sys.path.insert(0, '..')

import numpy as np
import matplotlib.pyplot as plt
from IPython.display import Audio, display, clear_output
import time

# Import BrainJam performance system
from performance_system.controllers import MockEEGController
from performance_system.sound_engines import ParametricSynth
from performance_system.mapping_models import LinearMapper

%matplotlib inline
plt.style.use('seaborn-v0_8-darkgrid')

## 1. Initialize Performer and AI Systems

We'll create two sound engines:
- **Performer voice**: Directly controlled by your input
- **AI voice**: Responds to and complements your input

In [ ]:
# Initialize controller (performer input)
controller = MockEEGController(fs=250)
print("✓ Performer Controller initialized")

# Initialize two synthesizers
performer_synth = ParametricSynth(sample_rate=44100, base_freq=220.0)  # A3
ai_synth = ParametricSynth(sample_rate=44100, base_freq=329.63)  # E4 (perfect fifth)

print("\n✓ Performer Voice (A3 base)")
print("✓ AI Voice (E4 base - perfect fifth harmony)")
print("\nThe AI voice will respond to and complement your input.")

## 2. Define AI Response Strategy

The AI co-performer uses a simple but musically meaningful strategy:

### Call-and-Response Pattern
- **When performer is active (high density)** → AI is sparse (listening)
- **When performer is sparse** → AI fills space (responding)

### Harmonic Relationship
- **When performer is consonant** → AI adds gentle harmony
- **When performer is tense** → AI mirrors tension

This creates a **musical dialogue**, not random accompaniment.

In [ ]:
class AICoPerformer:
    """
    AI co-performer that responds to performer input.
    
    Strategy:
    - Call-and-response: Active when performer is sparse, sparse when performer is active
    - Harmonic support: Follows performer's harmonic character
    - Temporal memory: Remembers recent performer behavior
    """
    
    def __init__(self, response_delay: float = 0.5):
        """
        Initialize AI co-performer.
        
        Args:
            response_delay: How long AI "listens" before responding (seconds)
        """
        self.response_delay = response_delay
        self.performer_history = []
        self.time_since_last = 0.0
        
    def generate_response(self, performer_controls: dict, dt: float) -> dict:
        """
        Generate AI response based on performer input.
        
        Args:
            performer_controls: Current performer control vector
            dt: Time since last call (seconds)
            
        Returns:
            AI control parameters
        """
        # Update history
        self.performer_history.append(performer_controls)
        if len(self.performer_history) > 10:  # Keep last 10 samples
            self.performer_history.pop(0)
        
        self.time_since_last += dt
        
        # Get performer's current state
        performer_density = performer_controls['control_1']
        performer_tension = performer_controls['control_2']
        performer_brightness = performer_controls['control_3']
        
        # Calculate average recent performer activity
        if len(self.performer_history) > 0:
            recent_density = np.mean([c['control_1'] for c in self.performer_history])
        else:
            recent_density = 0.5
        
        # AI STRATEGY 1: Call-and-response density
        # Inverse relationship: sparse when performer is dense
        ai_density = 1.0 - recent_density
        
        # Add some breathing room
        ai_density = ai_density * 0.6 + 0.2  # Scale to 0.2-0.8 range
        
        # AI STRATEGY 2: Harmonic support
        # Follow performer's harmonic character but slightly offset
        ai_tension = performer_tension * 0.7  # Slightly less tense
        
        # AI STRATEGY 3: Complementary brightness
        # Opposite brightness for textural contrast
        ai_brightness = 1.0 - performer_brightness
        
        # AI STRATEGY 4: Gentle noise texture
        ai_noise = 0.3  # Constant gentle texture
        
        return {
            'control_1': np.clip(ai_density, 0, 1),
            'control_2': np.clip(ai_tension, 0, 1),
            'control_3': np.clip(ai_brightness, 0, 1),
            'control_4': np.clip(ai_noise, 0, 1),
        }

# Initialize AI co-performer
ai_performer = AICoPerformer(response_delay=0.5)
print("✓ AI Co-Performer initialized")
print("\nAI Strategy:")
print("  1. Call-and-response: Fills space when you're sparse")
print("  2. Harmonic support: Follows your harmonic character")
print("  3. Textural contrast: Complements your brightness")
print("  4. Gentle presence: Adds subtle texture")

## 3. Generate Performer + AI Duet

Now let's hear the performer and AI together in a musical dialogue.

In [ ]:
# Reset everything
controller.reset()
performer_synth.reset()
ai_synth.reset()
ai_performer = AICoPerformer()

# Generate 10 seconds of duet
duration = 10.0
chunk_size = 0.1
n_chunks = int(duration / chunk_size)

performer_audio_chunks = []
ai_audio_chunks = []
performer_controls_log = []
ai_controls_log = []

print("Generating performer + AI duet...")

for i in range(n_chunks):
    # Get performer control
    performer_controls = controller.get_control_vector(duration=chunk_size)
    performer_controls_log.append(performer_controls)
    
    # AI responds to performer
    ai_controls = ai_performer.generate_response(performer_controls, chunk_size)
    ai_controls_log.append(ai_controls)
    
    # Generate audio for both
    performer_audio = performer_synth.generate(chunk_size, performer_controls)
    ai_audio = ai_synth.generate(chunk_size, ai_controls)
    
    performer_audio_chunks.append(performer_audio)
    ai_audio_chunks.append(ai_audio)
    
    if (i + 1) % 20 == 0:
        print(f"  Generated {(i + 1) * chunk_size:.1f}s / {duration}s")

# Concatenate audio
performer_audio = np.concatenate(performer_audio_chunks)
ai_audio = np.concatenate(ai_audio_chunks)

# Mix at appropriate levels
performer_level = 0.6
ai_level = 0.4
mixed_audio = performer_level * performer_audio + ai_level * ai_audio

print(f"\n✓ Generated {duration}s of duet")
print("\nListen to:")
print("  1. Performer voice only")
print("  2. AI voice only")
print("  3. Mixed duet")

print("\n--- PERFORMER VOICE ONLY ---")
display(Audio(performer_audio, rate=performer_synth.sample_rate))

print("\n--- AI VOICE ONLY ---")
display(Audio(ai_audio, rate=ai_synth.sample_rate))

print("\n--- MIXED DUET (60% Performer + 40% AI) ---")
display(Audio(mixed_audio, rate=performer_synth.sample_rate))

## 4. Visualize the Musical Dialogue

Let's see how performer and AI interact over time.

In [ ]:
# Extract control parameters
performer_density = [c['control_1'] for c in performer_controls_log]
performer_tension = [c['control_2'] for c in performer_controls_log]
ai_density = [c['control_1'] for c in ai_controls_log]
ai_tension = [c['control_2'] for c in ai_controls_log]

time_axis = np.arange(len(performer_density)) * chunk_size

# Plot
fig, axes = plt.subplots(2, 1, figsize=(14, 8))

# Density (Call-and-response)
axes[0].plot(time_axis, performer_density, 'b-', linewidth=2, label='Performer', alpha=0.8)
axes[0].plot(time_axis, ai_density, 'r--', linewidth=2, label='AI', alpha=0.8)
axes[0].set_ylabel('Tempo/Density', fontsize=11)
axes[0].set_ylim([0, 1])
axes[0].set_xlim([0, duration])
axes[0].legend(loc='upper right', fontsize=10)
axes[0].grid(True, alpha=0.3)
axes[0].set_title('Call-and-Response: AI is sparse when Performer is dense', 
                  fontsize=12, fontweight='bold')
axes[0].axhline(y=0.5, color='gray', linestyle=':', alpha=0.3)

# Fill areas to show inverse relationship
axes[0].fill_between(time_axis, 0, performer_density, alpha=0.2, color='blue', label='_nolegend_')
axes[0].fill_between(time_axis, 0, ai_density, alpha=0.2, color='red', label='_nolegend_')

# Harmonic tension
axes[1].plot(time_axis, performer_tension, 'b-', linewidth=2, label='Performer', alpha=0.8)
axes[1].plot(time_axis, ai_tension, 'r--', linewidth=2, label='AI', alpha=0.8)
axes[1].set_ylabel('Harmonic Tension', fontsize=11)
axes[1].set_ylim([0, 1])
axes[1].set_xlim([0, duration])
axes[1].set_xlabel('Time (seconds)', fontsize=11)
axes[1].legend(loc='upper right', fontsize=10)
axes[1].grid(True, alpha=0.3)
axes[1].set_title('Harmonic Support: AI follows Performer with gentle support', 
                  fontsize=12, fontweight='bold')
axes[1].axhline(y=0.5, color='gray', linestyle=':', alpha=0.3)

plt.suptitle('Performer–AI Musical Dialogue', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

print("\nObserve the musical dialogue:")
print("\n1. CALL-AND-RESPONSE (top):")
print("   - When Performer is dense (blue peaks) → AI is sparse (red valleys)")
print("   - When Performer is sparse (blue valleys) → AI fills space (red rises)")
print("\n2. HARMONIC SUPPORT (bottom):")
print("   - AI follows Performer's harmonic character")
print("   - AI provides gentler support (lower tension)")

## 5. Analyze Interaction Dynamics

Let's quantify the performer–AI relationship.

In [ ]:
# Calculate correlation between performer and AI
density_correlation = np.corrcoef(performer_density, ai_density)[0, 1]
tension_correlation = np.corrcoef(performer_tension, ai_tension)[0, 1]

# Calculate temporal offset (AI should respond slightly delayed)
performer_changes = np.diff(performer_density)
ai_changes = np.diff(ai_density)

# Measure responsiveness
performer_variance = np.var(performer_density)
ai_variance = np.var(ai_density)

print("="*60)
print("INTERACTION ANALYSIS")
print("="*60)

print("\n1. Density Correlation:")
print(f"   r = {density_correlation:.3f}")
if density_correlation < -0.3:
    print("   ✓ Strong inverse relationship (call-and-response working!)")
elif density_correlation > 0.3:
    print("   ⚠ Positive correlation (AI mimicking instead of responding)")
else:
    print("   ~ Weak relationship (AI relatively independent)")

print("\n2. Tension Correlation:")
print(f"   r = {tension_correlation:.3f}")
if tension_correlation > 0.5:
    print("   ✓ Strong positive relationship (harmonic support working!)")
elif tension_correlation < -0.3:
    print("   ⚠ Negative correlation (AI providing contrast)")
else:
    print("   ~ Moderate relationship")

print("\n3. Activity Levels:")
print(f"   Performer variance: {performer_variance:.3f}")
print(f"   AI variance: {ai_variance:.3f}")
print(f"   Activity ratio: {ai_variance/performer_variance:.2f}")

print("\n4. Interaction Pattern:")
performer_mean = np.mean(performer_density)
ai_mean = np.mean(ai_density)
print(f"   Performer average density: {performer_mean:.2f}")
print(f"   AI average density: {ai_mean:.2f}")
if abs(performer_mean + ai_mean - 1.0) < 0.2:
    print("   ✓ Balanced duet (complementary density levels)")

print("\n" + "="*60)
print("INTERPRETATION")
print("="*60)
print("\nThe AI is designed to be a responsive partner, not a mimic.")
print("- Negative density correlation = Call-and-response")
print("- Positive tension correlation = Harmonic support")
print("- Balanced activity = Musical dialogue, not dominance")
print("\n✓ This creates a musical conversation, not autonomous AI.")

## 6. Experiment: Different AI Response Strategies

Let's try different AI behaviors to compare:
1. **Call-and-response** (current)
2. **Mimic** (follow performer)
3. **Independent** (ignore performer)

In [ ]:
def generate_duet_with_strategy(strategy='call-response', duration=5.0):
    """
    Generate duet with different AI strategies.
    
    Args:
        strategy: 'call-response', 'mimic', or 'independent'
        duration: Length in seconds
    """
    # Reset
    controller.reset()
    performer_synth.reset()
    ai_synth.reset()
    
    chunk_size = 0.1
    n_chunks = int(duration / chunk_size)
    
    performer_chunks = []
    ai_chunks = []
    
    for i in range(n_chunks):
        # Performer control
        p_controls = controller.get_control_vector(duration=chunk_size)
        
        # AI strategy
        if strategy == 'call-response':
            # Inverse density, follow tension
            ai_controls = {
                'control_1': 1.0 - p_controls['control_1'] * 0.8 + 0.2,
                'control_2': p_controls['control_2'] * 0.7,
                'control_3': 1.0 - p_controls['control_3'],
                'control_4': 0.3,
            }
        elif strategy == 'mimic':
            # Follow performer exactly
            ai_controls = p_controls.copy()
        else:  # independent
            # Random/independent
            ai_controls = {
                'control_1': 0.5,
                'control_2': 0.5,
                'control_3': 0.5,
                'control_4': 0.3,
            }
        
        # Generate audio
        p_audio = performer_synth.generate(chunk_size, p_controls)
        ai_audio = ai_synth.generate(chunk_size, ai_controls)
        
        performer_chunks.append(p_audio)
        ai_chunks.append(ai_audio)
    
    performer_audio = np.concatenate(performer_chunks)
    ai_audio = np.concatenate(ai_chunks)
    mixed = 0.6 * performer_audio + 0.4 * ai_audio
    
    return mixed

print("Generating duets with different AI strategies...\n")

# Strategy 1: Call-and-response
print("1. CALL-AND-RESPONSE (Musical dialogue)")
print("   AI fills space when you're sparse")
audio1 = generate_duet_with_strategy('call-response')
display(Audio(audio1, rate=44100))

# Strategy 2: Mimic
print("\n2. MIMIC (AI copies performer)")
print("   AI follows your every move")
audio2 = generate_duet_with_strategy('mimic')
display(Audio(audio2, rate=44100))

# Strategy 3: Independent
print("\n3. INDEPENDENT (AI ignores performer)")
print("   AI does its own thing")
audio3 = generate_duet_with_strategy('independent')
display(Audio(audio3, rate=44100))

print("\n" + "="*60)
print("Which strategy sounds most musical?")
print("="*60)
print("\nCall-and-response typically creates the most engaging dialogue.")
print("The AI responds to you but maintains its own voice.")
print("\n✓ This is the goal: Musical interaction, not imitation or autonomy.")

## Summary

This notebook demonstrated:

1. ✓ **AI as co-performer** — Responsive musical partner, not autonomous
2. ✓ **Call-and-response** — AI fills space when performer is sparse
3. ✓ **Harmonic support** — AI follows performer's musical character
4. ✓ **Musical dialogue** — Interaction analysis shows complementary behavior
5. ✓ **Strategy comparison** — Call-and-response vs mimic vs independent

### Key Insights

- **AI is not autonomous** — It responds to your input
- **You lead, AI follows** — But with its own voice
- **Musical interaction** — Not random accompaniment
- **Performer agency** — You control the relationship

### Design Principles for AI Co-Performers

1. **Responsiveness** — AI reacts to performer in musically meaningful ways
2. **Complementarity** — AI adds to, doesn't duplicate, performer
3. **Transparency** — Relationship is understandable and predictable
4. **Controllability** — Performer can adjust AI behavior
5. **Musical coherence** — AI behavior is musically motivated

### Next Steps

- **Customize AI strategy** — Adjust response patterns to your preference
- **Add temporal memory** — AI remembers longer-term patterns
- **Multiple AI voices** — Create ensemble performances
- **Real-time parameter control** — Adjust AI responsiveness during performance
- **Perform live** — Try it with an audience!

---

**Remember**: The goal is **meaningful human–AI collaboration**,  
not replacing human creativity with autonomous AI.